In [1]:
import numpy as np
import os
import h5py
import pandas as pd
from time import time

from joblib import Parallel, delayed

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.decomposition import PCA

from LBP_components import Conv_MRELBP
from Components import local_normalize

import matplotlib.pyplot as plt

np.random.seed(42)

In [2]:
#Paths and files
path_surf = '../../cartvoi_surf_new/'
files_surf = os.listdir(path_surf)
files_surf.sort()
path_deep = '../../cartvoi_deep_new/'
files_deep = os.listdir(path_deep)
files_deep.sort()
path_calc = '../../cartvoi_calc_new/'
files_calc = os.listdir(path_calc)
files_calc.sort()

save_path = './results.csv'

#Image loading
im_surf = []
im_deep = []
im_calc = []

for fsurf,fdeep,fcalc in zip(files_surf,files_deep,files_calc):
    h5 = h5py.File(os.path.join(path_surf,fsurf),'r')
    ims = h5['sum'][:]
    h5.close()
    im_surf.append(ims)
    h5 = h5py.File(os.path.join(path_deep,fdeep),'r')
    imd = h5['sum'][24:-24,24:-24]
    h5.close()
    im_deep.append(imd)
    h5 = h5py.File(os.path.join(path_calc,fcalc),'r')
    imc = h5['sum'][24:-24,24:-24]
    h5.close()
    im_calc.append(imc)
    
#Grades
df = pd.read_excel('../../ERCGrades.xlsx')
sgrades = np.array(df['surf_sub'])
dgrades = np.array(df['deep_mat'])
cgrades = np.array(df['calc_mat'])

In [3]:
#Feature extration

args_surf = {'ks1': 9, 'sigma1': 3, 'ks2': 21, 'sigma2': 15, 'R': 18, 'r': 5, 'wc': 7, 'wR': 9, 'wr': 3}
args_deep = {'ks1': 25, 'sigma1': 12, 'ks2': 9, 'sigma2': 7, 'R': 27, 'r': 7, 'wc': 13, 'wR': 3, 'wr': 3}
args_calc = {'ks1': 11, 'sigma1': 11, 'ks2': 23, 'sigma2': 3, 'R': 3, 'r': 2, 'wc': 11, 'wR': 5, 'wr': 5}

features_surf = Parallel(n_jobs=12)(delayed(Conv_MRELBP)
                                    (local_normalize(img,args_surf['ks1'],args_surf['sigma1'],args_surf['ks2'],args_surf['sigma2']),
                                     8,args_surf['R'],args_surf['r'],args_surf['wR'],args_surf['wr'],args_surf['wc']) for img in im_surf)
features_deep = Parallel(n_jobs=12)(delayed(Conv_MRELBP)
                                    (local_normalize(img,args_deep['ks1'],args_deep['sigma1'],args_deep['ks2'],args_deep['sigma2']),
                                     8,args_deep['R'],args_deep['r'],args_deep['wR'],args_deep['wr'],args_deep['wc']) for img in im_deep)
features_calc = Parallel(n_jobs=12)(delayed(Conv_MRELBP)
                                    (local_normalize(img,args_calc['ks1'],args_calc['sigma1'],args_calc['ks2'],args_calc['sigma2']),
                                     8,args_calc['R'],args_calc['r'],args_calc['wR'],args_calc['wr'],args_calc['wc']) for img in im_calc)

features_surf = np.array(features_surf).squeeze()
features_deep = np.array(features_deep).squeeze()
features_calc = np.array(features_calc).squeeze()
#PCA
pcasurf = PCA(20,whiten=True,random_state=42)
surfpc = pcasurf.fit(features_surf).transform(features_surf)

pcadeep = PCA(20,whiten=True,random_state=42)
deeppc = pcadeep.fit(features_deep).transform(features_deep)

pcacalc = PCA(20,whiten=True,random_state=42)
calcpc = pcacalc.fit(features_calc).transform(features_calc)

In [4]:
#Regression

#Get splits
loo_surf = LeaveOneOut()
loo_surf.get_n_splits(surfpc)
loo_deep = LeaveOneOut()
loo_deep.get_n_splits(deeppc)
loo_calc = LeaveOneOut()
loo_calc.get_n_splits(calcpc)

#Evaluate surface
surfp = []
surfp_log = []
for train_idx, test_idx in loo_surf.split(surfpc):
    #Train split
    f = surfpc[train_idx]-surfpc.mean(0)
    g = sgrades[train_idx]
    
    #Linear regression
    Rmodel = Ridge(alpha=1,normalize=True,random_state=42)
    Rmodel.fit(f,g.reshape(-1,1))
    
    #Logistic regression
    Lmodel = LogisticRegression(solver='newton-cg',max_iter=1000)
    Lmodel.fit(f,g.ravel()>=1)
    
    #Evaluate on test sample
    p = Rmodel.predict((surfpc[test_idx]-surfpc.mean(0)).reshape(1,-1))
    p_log = Lmodel.predict_proba((surfpc[test_idx]-surfpc.mean(0)).reshape(1,-1))
    surfp.append(p)
    surfp_log.append(p_log)

#Evaluate deep cartilage
deepp = []
deepp_log = []
for train_idx, test_idx in loo_deep.split(deeppc):
    #Train split
    f = deeppc[train_idx]-deeppc.mean(0)
    g = dgrades[train_idx]
    
    #Linear regression
    Rmodel = Ridge(alpha=1,normalize=True,random_state=42)
    Rmodel.fit(f,g.reshape(-1,1))
    
    #Logistic regression
    Lmodel = LogisticRegression(solver='newton-cg',max_iter=1000)
    Lmodel.fit(f,g.ravel()>=1)
    
    #Evaluate on test sample
    p = Rmodel.predict((deeppc[test_idx]-deeppc.mean(0)).reshape(1,-1))
    p_log = Lmodel.predict_proba((deeppc[test_idx]-deeppc.mean(0)).reshape(1,-1))
    deepp.append(p)
    deepp_log.append(p_log)
    
#Evaluate calcified cartilage
calcp = []
calcp_log = []
for train_idx, test_idx in loo_surf.split(calcpc):
    #Train split
    f = calcpc[train_idx]-calcpc.mean(0)
    g = cgrades[train_idx]
    
    #Linear regression
    Rmodel = Ridge(alpha=1,normalize=True,random_state=42)
    Rmodel.fit(f,g.reshape(-1,1))
    
    #Logistic regression
    Lmodel = LogisticRegression(solver='newton-cg',max_iter=1000)
    Lmodel.fit(f,g.ravel()>=1)
    
    #Evaluate on test sample
    p = Rmodel.predict((calcpc[test_idx]-calcpc.mean(0)).reshape(1,-1))
    p_log = Lmodel.predict_proba((calcpc[test_idx]-calcpc.mean(0)).reshape(1,-1))
    calcp.append(p)
    calcp_log.append(p_log)


In [5]:
#Predictions to array
surfp = np.array(surfp).squeeze()
surfp_log = np.array(surfp_log).squeeze()
deepp = np.array(deepp).squeeze()
deepp_log = np.array(deepp_log).squeeze()
calcp = np.array(calcp).squeeze()
calcp_log = np.array(calcp_log).squeeze()

In [6]:
def MSE(preds,targets):
    N = len(preds)
    errors = preds.flatten()-targets.flatten()
    return (errors**2).sum()/N

print(MSE(surfp,sgrades))
print(MSE(deepp,dgrades))
print(MSE(calcp,cgrades))

0.33857244702316813
0.5282088973272382
0.6272411202435833


In [7]:
print(np.concatenate((cgrades.reshape(-1,1),calcp.reshape(-1,1)),1))

[[ 1.          0.89550347]
 [ 1.          1.32920342]
 [ 2.          1.64509045]
 [ 1.          1.22640738]
 [ 0.          0.32151007]
 [ 1.          1.21410149]
 [ 1.          1.69864395]
 [ 0.          0.77343705]
 [ 0.          0.42294304]
 [ 0.          1.10477738]
 [ 3.          1.28103776]
 [ 2.          1.39362644]
 [ 1.          1.44051993]
 [ 1.          1.48727063]
 [ 2.          1.09515781]
 [ 1.          0.90440449]
 [ 1.          0.71261547]
 [ 0.          0.84543369]
 [ 3.          1.47212901]
 [ 1.          1.84806494]
 [ 1.          1.24245341]
 [ 0.          0.77026994]
 [ 2.          2.23368815]
 [ 3.          2.04008358]
 [ 2.          0.68304641]
 [ 1.          0.89173296]
 [ 0.          0.63552869]
 [ 0.          0.82899927]
 [ 2.          1.43461266]
 [ 1.          0.39175041]
 [ 1.          1.41480219]
 [ 1.         -1.16497929]
 [ 1.          0.63797873]
 [ 1.          1.31857991]
 [ 2.          1.02973506]
 [ 1.          0.83199163]]


In [8]:
dgrades.shape

(36,)